## Demo of buoyancy driven stokes flow in a Spherical Cap

In [1]:
import underworld as uw
import glucifer
import numpy as np
from underworld import function as fn

/Users/lmoresi/+Underworld/underworld2c/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# boundary conditions available "BC_FREESLIP, "BC_NOSLIP, "BC_LIDDRIVEN"
bc_wanted = 'BC_FREESLIP'

In [3]:
mesh = uw.mesh.FeMesh_SphericalCap(elementRes=(12,24,24), cap_size=(0.5,1.0, 180.0), elementType="Q1/dQ0")

TField = mesh.add_variable(nodeDofCount=1)
vField = mesh.add_variable(nodeDofCount=mesh.dim)
pField = mesh.subMesh.add_variable(nodeDofCount=1)

bodyForce = mesh.add_variable(nodeDofCount=mesh.dim)

core          = mesh.specialSets["core_VertexSet"]
surface       = mesh.specialSets["surface_VertexSet"]
wall          = mesh.specialSets["wall_VertexSet"]
allBoundaries = core+surface+wall

In [4]:
# create checkpoint function
def checkpoint( mesh, fieldDict, swarm, swarmDict, index,
                meshName='mesh', swarmName='swarm', 
                prefix='./', enable_xdmf=True):
    import os
    # Check the prefix is valid
    if prefix is not None:
        if not prefix.endswith('/'): prefix += '/' # add a backslash
        if not os.path.exists(prefix) and uw.rank()==0:
            print "Creating directory: ",prefix 
            os.makedirs(prefix)
        uw.barrier() 
            
    if not isinstance(index, int):
        raise TypeError("'index' is not of type int")        
    ii = str(index)
    
    if mesh is not None:
        
        # Error check the mesh and fields
        if not isinstance(mesh, uw.mesh.FeMesh):
            raise TypeError("'mesh' is not of type uw.mesh.FeMesh")
        if not isinstance(fieldDict, dict):
            raise TypeError("'fieldDict' is not of type dict")
        for key, value in fieldDict.iteritems():
            if not isinstance( value, uw.mesh.MeshVariable ):
                raise TypeError("'fieldDict' must contain uw.mesh.MeshVariable elements")


        # see if we have already saved the mesh. It only needs to be saved once
        if not hasattr( checkpoint, 'mH' ):
            checkpoint.mH = mesh.save(prefix+meshName+".h5")
        mh = checkpoint.mH

        for key,value in fieldDict.iteritems():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, mh, meshName)
        
    # is there a swarm
    if swarm is not None:
        
        # Error check the swarms
        if not isinstance(swarm, uw.swarm.Swarm):
            raise TypeError("'swarm' is not of type uw.swarm.Swarm")
        if not isinstance(swarmDict, dict):
            raise TypeError("'swarmDict' is not of type dict")
        for key, value in swarmDict.iteritems():
            if not isinstance( value, uw.swarm.SwarmVariable ):
                raise TypeError("'fieldDict' must contain uw.swarm.SwarmVariable elements")
    
        sH = swarm.save(prefix+swarmName+"-"+ii+".h5")
        for key,value in swarmDict.iteritems():
            filename = prefix+key+'-'+ii
            handle = value.save(filename+'.h5')
            if enable_xdmf: value.xdmf(filename, handle, key, sH, swarmName)


In [5]:
# zero all dofs of vField
vField.data[...] = 0.

if bc_wanted == "BC_NOSLIP":
    # no slip
    vBC = uw.conditions.RotatedDirichletCondition( variable=vField, 
                                                  indexSetsPerDof=(allBoundaries,allBoundaries,allBoundaries),
                                                  basis_vectors = (mesh._e1, mesh._e2, mesh._e3)
                                                 )

elif bc_wanted == "BC_FREESLIP":
    # free-slip
    
    vBC = uw.conditions.RotatedDirichletCondition( variable=vField,
                                                   indexSetsPerDof=(core+surface,None,wall),
                                                   basis_vectors = (mesh._e1, mesh._e2, mesh._e3) )
elif bc_wanted == "BC_LIDDRIVEN":
    # lid-driven case
        
    # build driving node indexset & apply velocities with zero radial component
    drivers = outer - (N+S+E+W)
    vField.data[drivers.data] = (0.,1.,1.)
    
    # build corner edges node indexset and apply velocities with zero non-radial components
    cEdge = (N&W)+(N&E)+(S&E)+(S&W)
    vField.data[cEdge.data] = (0.,0.,0.)
    
    # apply altogether.
    NS0 = N+S - (E+W)
    vBC = uw.conditions.RotatedDirichletCondition( variable=vField,
                                                  indexSetsPerDof=(inner+outer,drivers+E+W+cEdge,drivers+NS0+cEdge), # optional, can include cEdge on the 3rd component
                                                  basis_vectors = (mesh._e1, mesh._e2, mesh._e3) )
else:
    raise ValueError("Can't find an option for the 'bc_wanted' = {}".format(bc_wanted))

In [6]:
z_hat  = mesh.fn_unitvec_r()

In [7]:
(r, theta, phi) = mesh.fn_r_theta_phi()
temperatureFn = fn.math.cos(theta)
TField.data[:] = temperatureFn.evaluate(mesh) # For checkpointing

In [8]:
bodyForceFn = temperatureFn * z_hat
bodyForce.data[:] = bodyForceFn.evaluate(mesh) # For checkpointing 

In [9]:
# xdmf output
fieldDict = {'velocity':vField,
             'phi':mesh._e2,
             'radial':mesh._e1,
             'theta':mesh._e3,
             'temperature':TField,
             'buoyancy':bodyForce}
checkpoint(mesh, fieldDict, None, None, index=0, prefix='outputHemi')

In [10]:
stokesSLE = uw.systems.Stokes( vField, pField, 
                               fn_viscosity=1.0, fn_bodyforce=bodyForceFn, 
                               conditions=vBC,  _removeBCs=False)

In [11]:
stokesSolver = uw.systems.Solver(stokesSLE)
# if uw.nProcs() == 1:
#     stokesSolver.set_inner_method("lu")
    
# stokesSolver.set_inner_method("mumps")
# stokesSolver.set_penalty(10000.0)
# stokesSolver.options.main.restore_K=False
# stokesSolver.options.main.Q22_pc_type="gkgdiag"
# stokesSolver.options.A11.ksp_monitor="ascii"
# stokesSolver.options.scr.ksp_monitor="ascii"
# stokesSolver.options.rhsA11.ksp_monitor="ascii"


stokesSolver.options.main.Q22_pc_type="gkgdiag"
stokesSolver.options.A11.ksp_type="fgmres"
stokesSolver.options.scr.ksp_rtol=1.0e-3
stokesSolver.options.scr.ksp_monitor="ascii"
stokesSolver.options.A11.ksp_rtol=1.0e-4
stokesSolver.options.A11.ksp_monitor="ascii"
stokesSolver.set_penalty(1000.0)
stokesSolver.options.main.restore_K=False
stokesSolver.options.mg_accel.mg_accelerating_smoothing=True
stokesSolver.options.mg_accel.mg_smooths_to_start=1
stokesSolver.options.mg_accel.mg_smooths_max=10
stokesSolver.options.mg.mg_levels_ksp_convergence_test='skip'
stokesSolver.options.mg.mg_levels_ksp_max_its=3 
stokesSolver.options.mg.mg_coarse_pc_type="lu"
stokesSolver.options.mg.mg_coarse_pc_factor_mat_solver_package="mumps"
# solver.options.mg.mg_coarse_ksp_view="ascii" # to check the above



In [12]:
stokesSolver.solve()

In [13]:
uw.libUnderworld.Underworld.AXequalsX( stokesSLE._rot._cself, stokesSLE._velocitySol._cself, False)

In [14]:
stokesSolver.print_stats()


 
Pressure iterations:   6
Velocity iterations:  51 (presolve)      
Velocity iterations: 128 (pressure solve)
Velocity iterations:  59 (backsolve)     
Velocity iterations: 238 (total solve)   
 
SCR RHS  solve time: 2.1922e+00
Pressure solve time: 2.9225e+00
Velocity solve time: 2.4451e+00 (backsolve)
Total solve time   : 7.7099e+00
 
Velocity solution min/max: 0.0000e+00/0.0000e+00
Pressure solution min/max: 0.0000e+00/0.0000e+00
 



In [15]:
vdotv = fn.math.dot(vField,vField)
vrms = np.sqrt( mesh.integrate(vdotv)[0] / mesh.integrate(1.)[0] )
if uw.rank() == 0:
    print("The vrms = {:.5e}\n".format(vrms))

The vrms = 1.72924e-03



In [16]:
figV = glucifer.Figure()
figV.append(glucifer.objects.Mesh(mesh, segmentsPerEdge=3, opacity=0.25))
# figV.append(glucifer.objects.Surface(mesh, vdotv, onMesh=True))
# figV.append( glucifer.objects.VectorArrows(mesh, vField, autoscale=True, onMesh=True, glyphs=1))
figV.window()

<IPython.core.display.Javascript object>

{}

In [17]:
# xdmf output
fieldDict = {'velocity':vField,
             'temperature':TField}
checkpoint(mesh, fieldDict, None, None, index=0, prefix='outputHemi')